#Tutorial and EDA of 120 years of olympic Games
Lo que quiero hacer es un análisis exploratorio y visualización de datos de 120 años de los juegos olímpicos con el objetivo de identificar que variable puedo estimar utilizando algun modelo predictivo, de preferencia con alguna técnica de machine learning.

Content
The file athlete_events.csv contains 271116 rows and 15 columns. Each row corresponds to an individual athlete competing in an individual Olympic event (athlete-events). The columns are:

- ID - Unique number for each athlete
- Name - Athlete's name
- Sex - M or F
- Age - Integer
- Height - In centimeters
- Weight - In kilograms
- Team - Team name
- NOC - National Olympic Committee 3-letter code
- Games - Year and season
- Year - Integer
- Season - Summer or Winter
- City - Host city
- Sport - Sport
- Event - Event
- Medal - Gold, Silver, Bronze, or NA

In [86]:
#dataframes and data manipulation
import pandas as pd
import numpy as np
#data visualization
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import math
#sklearn to build ML models
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
# To encode categorical variables
from sklearn.preprocessing import LabelEncoder
#for hyperparameter tunning
from sklearn.model_selection import GridSearchCV
#for evaluation of the models
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
    classification_report,
    precision_recall_curve,
    roc_curve,
    make_scorer,
)

import warnings
warnings.filterwarnings("ignore")

#from bubbly.bubbly import bubbleplot
#from __future__ import division
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
#mount google drive to access the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [88]:
# read the data from google drive
dataf = pd.read_csv('/content/drive/MyDrive/DSPM_MIT/Projects/Olympic Games/Olympic_Data.csv')
dataf2= pd.read_csv('/content/drive/MyDrive/DSPM_MIT/Projects/Olympic Games/Noc_Data.csv')
df_olymp=dataf.copy()
df_noc=dataf2.copy()

In [89]:
df_olymp.shape

(271116, 15)

In [90]:
print(f'The data has {df_olymp.shape[0]} rows and {df_olymp.shape[1]} columns')

The data has 271116 rows and 15 columns


In [91]:
# head of the data
df_olymp.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [92]:
# head of the data
df_noc.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [93]:
#Checking the data types
df_olymp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [94]:
df_noc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NOC     230 non-null    object
 1   region  227 non-null    object
 2   notes   21 non-null     object
dtypes: object(3)
memory usage: 5.5+ KB


La columna de ID no me sirve de nada, tenemos 10 variables categoricas y 4 numericas sin contar el ID. La colimna de Games es la combinacion de Year y Season, por lo que Games tambien la podemos eliminar.

df_olymp tiene varias columnas sin valores. Hay que identificar si podemos tratarlos o eliminarlos. Lo mismo con el df_noc, una de las columnas tiene 90% de missing values por lo que la puedo eliminar.

In [95]:
df_olymp.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,271116.0,68248.954396,39022.286345,1.0,34643.0,68205.0,102097.25,135571.0
Age,261642.0,25.556898,6.393561,10.0,21.0,24.0,28.00,97.0
Height,210945.0,175.338970,10.518462,127.0,168.0,175.0,183.00,226.0
Weight,208241.0,70.702393,14.348020,25.0,60.0,70.0,79.00,214.0
Year,271116.0,1978.378480,29.877632,1896.0,1960.0,1988.0,2002.00,2016.0


De todos estas variables, vamos a definir cual seria nuestra variable dependiente e independiente, es decir, que variable quiero predecri de todos estos datos? La única opcion lógica es las medallas. Y las posibles relaciones que me pueden interesar son:
- ¿Quienes ganaron mas medallas?
- ¿algo con el genero?
- ¿Existe algun rango de edad el cual gane mas medallas?
- ¿promedio de altura? ¿promedio de peso? ¿promedio de peso que gano oro/plata/bronce?
- ¿cuantos equipos hay?¿que equipos ganaron mas?
- noc es un codigo que representa la region, puedo cambiar el codigo por la region para que sea mas facil de entender, y despues determinar ¿que region gano mas medallas?¿que region tiene mas equipos?¿mas hombres/mujeres? etc.
- de que me sirve el año? para identificar en que año gano quien, que equipo y region, promedio de peso/altura por año,etc...
- lo mismo para la temporada, identificar en que año gano quien, que equipo y region, promedio de peso/altura por season, etc...
- Que ciudades han sido hosts de los juegos?
- Cuantos deportes se han jugado? por año y temporada?
- Cuantos eventos hay? que tipos de eventos son?
- Quien ha ganado mas medallas? Medallas en hombres? Medallas en mujeres? Medallas por edad? Medallas por peso/altura? Medallas por equipos? Medallas por Region? algo con el años y la temporada? Medallas por evento y deporte?

## Vamo a calmarno

Primero vamos a eliminar las variables que no necesitamos.

In [11]:
#byebye Games
df_olymp=df_olymp.drop('ID', axis=1)

In [96]:
#byebye Games
df_olymp=df_olymp.drop('Games', axis=1)

In [97]:
df_olymp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 14 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Year    271116 non-null  int64  
 9   Season  271116 non-null  object 
 10  City    271116 non-null  object 
 11  Sport   271116 non-null  object 
 12  Event   271116 non-null  object 
 13  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 29.0+ MB


In [98]:
df_olymp.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,271116.0,68248.954396,39022.286345,1.0,34643.0,68205.0,102097.25,135571.0
Age,261642.0,25.556898,6.393561,10.0,21.0,24.0,28.00,97.0
Height,210945.0,175.338970,10.518462,127.0,168.0,175.0,183.00,226.0
Weight,208241.0,70.702393,14.348020,25.0,60.0,70.0,79.00,214.0
Year,271116.0,1978.378480,29.877632,1896.0,1960.0,1988.0,2002.00,2016.0


Ahora si, separamos las variables
- Numéricas: Age, Weight, Height, ¿Year?
- Categóricas: Name, Sex, Team, NOC, Season, City, Sport, Event, Medal(target).

In [99]:
# dividing categorical columns and numerical columns
cat_col = list(df_olymp.select_dtypes("object").columns)#categorical variables
num_col = list(df_olymp.select_dtypes("number").columns)#numerical variables
cols= list(df_olymp.columns)#all columns in list
cat_col_=['Name', 'Sex', 'Team', 'NOC', 'Year', 'Season', 'City', 'Sport', 'Event', 'Medal']

In [100]:
print(num_col)

['ID', 'Age', 'Height', 'Weight', 'Year']


In [17]:
num_col.pop(-1)

'Year'

In [101]:
# value counts for each categorical column in pctg
for column in cat_col_:
    print(f'{column}')
    print(df_olymp[column].value_counts())
    print("----" * 10)

Name
Name
Robert Tait McKenzie            58
Heikki Ilmari Savolainen        39
Joseph "Josy" Stoffel           38
Ioannis Theofilakis             36
Takashi Ono                     33
                                ..
Tatyana Vasilyevna Kalmykova     1
Mariya Lvovna Kalmykova          1
Christine Kalmer                 1
Joannis "Jannis" Kalmazidis      1
Pierre-Georges LeClercq          1
Name: count, Length: 134732, dtype: int64
----------------------------------------
Sex
Sex
M    196594
F     74522
Name: count, dtype: int64
----------------------------------------
Team
Team
United States    17847
France           11988
Great Britain    11404
Italy            10260
Germany           9326
                 ...  
Briar                1
Hannover             1
Nan-2                1
Brentina             1
Digby                1
Name: count, Length: 1184, dtype: int64
----------------------------------------
NOC
NOC
USA    18853
FRA    12758
GBR    12256
ITA    10715
GER     9830
      

In [102]:
df_olymp.isnull().sum()*100/df_olymp.shape[0]

ID         0.000000
Name       0.000000
Sex        0.000000
Age        3.494445
Height    22.193821
Weight    23.191180
Team       0.000000
NOC        0.000000
Year       0.000000
Season     0.000000
City       0.000000
Sport      0.000000
Event      0.000000
Medal     85.326207
dtype: float64

In [103]:
df_olymp['Team'].nunique()

1184

In [104]:
df_olymp['NOC'].nunique()

230

In [105]:
df_olymp.groupby("NOC")['Team'].nunique()

NOC
AFG    1
AHO    1
ALB    1
ALG    1
AND    1
      ..
YEM    1
YMD    1
YUG    6
ZAM    1
ZIM    2
Name: Team, Length: 230, dtype: int64

Hay 1189 equipos y solamente 230 regiones, por lo tanto no puedo eliminar la columna de NOC, pero si podemos preguntarno ¿cuantos equipos hay en cada region? y eso tiene alguna influencia en la cantidad de medallas?


In [106]:
#Mean of the numerical columns
for column in num_col:
    print(f'Average of {column} is :')
    print(df_olymp[column].mean())
    print("----" * 10)

Average of ID is :
68248.95439590434
----------------------------------------
Average of Age is :
25.556898357297374
----------------------------------------
Average of Height is :
175.33896987366376
----------------------------------------
Average of Weight is :
70.70239290053351
----------------------------------------
Average of Year is :
1978.3784800601957
----------------------------------------


El promedio es una mala forma de medir porque estamos tomando en cuenta todos los deporte, pero podriamos agrupar el la altura/peso con los deportes/eventos y obtener el promedio.

In [108]:
HW=['Height', 'Weight']

In [109]:
df_olymp.groupby("Sport")[HW].mean()

,Height,Weight
Sport,,
Aeronautics,NaN,NaN
Alpine Skiing,173.489052,72.068110
Alpinism,NaN,NaN
Archery,173.203085,70.011135
Art Competitions,174.644068,75.290909
...,...,...
Tug-Of-War,182.480000,95.615385
Volleyball,186.994822,78.900214
Water Polo,184.834648,84.566446


In [110]:
df_olymp.groupby("Event")[HW].mean()

,Height,Weight
Event,,
Aeronautics Mixed Aeronautics,NaN,NaN
Alpine Skiing Men's Combined,180.143939,83.737245
Alpine Skiing Men's Downhill,177.947984,79.260756
Alpine Skiing Men's Giant Slalom,176.820246,76.398104
Alpine Skiing Men's Slalom,177.275732,76.591383
...,...,...
"Wrestling Women's Flyweight, Freestyle",156.014706,48.661765
"Wrestling Women's Heavyweight, Freestyle",170.890625,73.296875
"Wrestling Women's Light-Heavyweight, Freestyle",169.277778,69.666667


## Events and Cities.

In [111]:
tmp = df_olymp.groupby(['Year', 'City'])['Season'].value_counts()
df_EV = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()

In [112]:
df_EV.head()

,Year,City,Season,Athlets
0,1896,Athina,Summer,380
1,1900,Paris,Summer,1936
2,1904,St. Louis,Summer,1301
3,1906,Athina,Summer,1733
4,1908,London,Summer,3101


In [113]:
trace = go.Scatter(
    x = df_EV['Year'],
    y = df_EV['Athlets'],
    name="Athlets per Olympic game",
    marker=dict(
        color="Blue",
    ),
    mode = "markers"
)
data = [trace]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets1')

In [114]:
dfS = df_EV[df_EV['Season']=='Summer']; dfW = df_EV[df_EV['Season']=='Winter']

traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    mode = "markers+lines"
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    mode = "markers+lines"
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets2')

In [115]:
traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    mode = "markers+lines",
    text=dfS['City'],
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    mode = "markers+lines",
    text=dfW['City']
)

data = [traceS, traceW]

fig = make_subplots(rows=1, cols=2, subplot_titles=('Number athlets: Summer Games', 'Number athlets: Winter Games'))
fig.append_trace(traceS, 1, 1)
fig.append_trace(traceW, 1, 2)

iplot(fig, filename='events-athlets2')

Se pueden ver el numero de atletas en cada año, y dividido por temporada de invierno y de verano.

In [116]:
tmp = df_olymp.groupby('Year')['City'].value_counts()
df2 = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
df2 = df2.merge(df_EV)

In [117]:
df2.head()

,Year,City,Athlets,Season
0,1896,Athina,380,Summer
1,1900,Paris,1936,Summer
2,1904,St. Louis,1301,Summer
3,1906,Athina,1733,Summer
4,1908,London,3101,Summer


In [118]:
iplot(ff.create_table(df2.head()), filename='jupyter-table2')

In [119]:
dfS = df2[df2['Season']=='Summer']; dfW = df2[df2['Season']=='Winter']

traceS = go.Bar(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
    marker=dict(color="Red"),
    text=dfS['City']
)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(color="Blue"),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets3')

In [120]:
traceS = go.Bar(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='black',
                    width=0.75),
                opacity=0.7,
            ),
    text=dfS['City'],

)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='black',
                    width=0.75),
                opacity=0.7,
            ),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of athlets'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets4')

Con estas graficas podemos visualizar el numero de atletas por cada temporada, la ciudad donde se llevaron acabo los juegos y el año. 1992 fue el ultimo año donde los juegos de verano y de invierno se llevaron acabo en el mismo años, a partir de 1994 los juegos se organizaron en años diferentes, cada 2 años.

In [121]:
traceS = go.Box(
    x = dfS['Athlets'],
    name="Summer Games",

     marker=dict(
                color='rgba(238,23,11,0.5)',
                line=dict(
                    color='red',
                    width=1.2),
            ),
    text=dfS['City'],
    orientation='h',

)
traceW = go.Box(
    x = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgba(11,23,245,0.5)',
                line=dict(
                    color='blue',
                    width=1.2),
            ),
    text=dfS['City'],  orientation='h',
)

data = [traceS, traceW]
layout = dict(title = 'Athlets per Olympic game',
          xaxis = dict(title = 'Number of athlets',showticklabels=True),
          yaxis = dict(title = 'Season', showticklabels=True, tickangle=-90),
          hovermode = 'closest',
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-athlets5')

## SPORTS

In [122]:
tmp = df_olymp.groupby(['Year', 'City','Season'])['Sport'].nunique()
df_s = pd.DataFrame(data={'Sports': tmp.values}, index=tmp.index).reset_index()

In [123]:
dfS = df_s[df_s['Season']=='Summer']; dfW = df_s[df_s['Season']=='Winter']

traceS = go.Bar(
    x = dfS['Year'],y = dfS['Sports'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='red',
                    width=1),
                opacity=0.5,
            ),
    text= dfS['City'],
)
traceW = go.Bar(
    x = dfW['Year'],y = dfW['Sports'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='blue',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['City']
)

data = [traceS, traceW]
layout = dict(title = 'Sports per Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of sports'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

In [124]:
tmp = df_olymp.groupby(['Year', 'City','Season'])['Sport'].value_counts()
df_s = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()

In [125]:
dfS = df_s[df_s['Season']=='Summer']; dfW = df_s[df_s['Season']=='Winter']


traceS = go.Scatter(
    x = dfS['Year'],y = dfS['Athlets'],
    name="Summer Games",
     marker=dict(
                color='rgb(238,23,11)',
                line=dict(
                    color='red',
                    width=1),
                opacity=0.5,
            ),
    text= "City:"+dfS['City']+" Sport:"+dfS['Sport'],
    mode = "markers"
)
traceW = go.Scatter(
    x = dfW['Year'],y = dfW['Athlets'],
    name="Winter Games",
    marker=dict(
                color='rgb(11,23,245)',
                line=dict(
                    color='blue',
                    width=1),
                opacity=0.5,
            ),
   text= "City:"+dfW['City']+" Sport:"+dfW['Sport'],
    mode = "markers"
)

data = [traceS, traceW]
layout = dict(title = 'Number of athlets per sport for each Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of athlets per sport'),
          hovermode='closest'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

El deporte con mas atletas es el atletismo. JAJA

In [126]:
tmp = df_olymp.groupby(['Year', 'City','Season'])['Sport'].value_counts()
df = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()

In [127]:
sports = (df_olymp.groupby(['Sport'])['Sport'].nunique()).index

In [128]:
def draw_trace(dataset, sport):
    dfS = dataset[dataset['Sport']==sport];
    trace = go.Box(
        x = dfS['Athlets'],
        name=sport,
         marker=dict(
                    line=dict(
                        color='black',
                        width=0.8),
                ),
        text=dfS['City'],
        orientation = 'h'
    )
    return trace


def draw_group(dataset, title,height=800):
    data = list()
    for sport in sports:
        data.append(draw_trace(dataset, sport))


    layout = dict(title = title,
              xaxis = dict(title = 'Number of athlets',showticklabels=True),
              yaxis = dict(title = 'Sport', showticklabels=True, tickfont=dict(
                family='Old Standard TT, serif',
                size=8,
                color='black'),),
              hovermode = 'closest',
              showlegend=False,
                  width=800,
                  height=height,
             )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='events-sports1')

In [129]:
# select only Summer Olympics
df_S = df[df['Season']=='Summer']
# draw the boxplots for the Summer Olympics
draw_group(df_S, "Athlets per Sport (Summer Olympics)")

In [130]:
# select only Winter Olympics
df_W = df[df['Season']=='Winter']
# draw the boxplots for the Summer Olympics
draw_group(df_W, "Athlets per Sport (Winter Olympics)",600)

## COUNTRIES

In [131]:
olympics_df = df_olymp.merge(df_noc)
olympics_df.shape

(270767, 16)

In [132]:
olympics_df=olympics_df.rename(columns = {'region':'Country'})

In [133]:
tmp = olympics_df.groupby(['Country'])['Year'].nunique()
df = pd.DataFrame(data={'Editions': tmp.values}, index=tmp.index).reset_index()
df.head(2)

,Country,Editions
0,Afghanistan,14
1,Albania,11


In [134]:
trace = go.Choropleth(
            locations = df['Country'],
            locationmode='country names',
            z = df['Editions'],
            text = df['Country'],
            autocolorscale =False,
            reversescale = True,
            colorscale = 'rainbow',
            marker = dict(
                line = dict(
                    color = 'rgb(0,0,0)',
                    width = 0.5)
            ),
            colorbar = dict(
                title = 'Editions',
                tickprefix = '')
        )

data = [trace]
layout = go.Layout(
    title = 'Olympic countries',
    geo = dict(
        showframe = True,
        showlakes = False,
        showcoastlines = True,
        projection = dict(
            type = 'natural earth'
        )
    )
)

fig = dict( data=data, layout=layout )
iplot(fig)

In [135]:
tmp = olympics_df.groupby(['Country', 'Season'])['Year'].nunique()
df = pd.DataFrame(data={'Editions': tmp.values}, index=tmp.index).reset_index()
df.head(2)

,Country,Season,Editions
0,Afghanistan,Summer,14
1,Albania,Summer,8


In [136]:
dfS = df[df['Season']=='Summer']; dfW = df[df['Season']=='Winter']

def draw_map(dataset, title, colorscale, reversescale=False):
    trace = go.Choropleth(
                locations = dataset['Country'],
                locationmode='country names',
                z = dataset['Editions'],
                text = dataset['Country'],
                autocolorscale =False,
                reversescale = reversescale,
                colorscale = colorscale,
                marker = dict(
                    line = dict(
                        color = 'rgb(0,0,0)',
                        width = 0.5)
                ),
                colorbar = dict(
                    title = 'Editions',
                    tickprefix = '')
            )

    data = [trace]
    layout = go.Layout(
        title = title,
        geo = dict(
            showframe = True,
            showlakes = False,
            showcoastlines = True,
            projection = dict(
                type = 'orthographic'
            )
        )
    )
    fig = dict( data=data, layout=layout )
    iplot(fig)


In [137]:
draw_map(dfS, 'Olympic countries (Summer games)', "Reds")

In [138]:
draw_map(dfW, 'Olympic countries (Winter games)', "Blues", True)

In [139]:
tmp = olympics_df.groupby(['Year','Sport'])['Country'].value_counts()
dataset = pd.DataFrame(data={'Athlets': tmp.values}, index=tmp.index).reset_index()
dataset.head()

,Year,Sport,Country,Athlets
0,1896,Athletics,Greece,36
1,1896,Athletics,USA,21
2,1896,Athletics,Germany,14
3,1896,Athletics,France,12
4,1896,Athletics,UK,7


## ATHLETS

In [140]:
female_h = olympics_df[olympics_df['Sex']=='F']['Height'].dropna()
male_h = olympics_df[olympics_df['Sex']=='M']['Height'].dropna()

hist_data = [female_h, male_h]
group_labels = ['Female Height', 'Male Height']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Height distribution plot')
iplot(fig, filename='dist_only')

In [141]:
female_w = olympics_df[olympics_df['Sex']=='F']['Weight'].dropna()
male_w = olympics_df[olympics_df['Sex']=='M']['Weight'].dropna()

hist_data = [female_w, male_w]
group_labels = ['Female Weight', 'Male Weight']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Weight distribution plot')
iplot(fig, filename='dist_only')

In [142]:
female_a = olympics_df[olympics_df['Sex']=='F']['Age'].dropna()
male_a = olympics_df[olympics_df['Sex']=='M']['Age'].dropna()

hist_data = [female_a, male_a]
group_labels = ['Female Age', 'Male Age']

fig = ff.create_distplot(hist_data, group_labels, show_hist=False, show_rug=False)
fig['layout'].update(title='Athlets Age distribution plot')
iplot(fig, filename='dist_only')

In [143]:
tmp_1 = olympics_df.groupby(['Sport'])[HW].agg('mean').dropna()
df_1 = pd.DataFrame(tmp_1).reset_index()
tmp_2 = olympics_df.groupby(['Sport'])['ID'].count()
df_2 = pd.DataFrame(tmp_2).reset_index()
dataset = df_1.merge(df_2)

In [144]:
#Hover text
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sport: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Sport'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [145]:
#create Bubble scatter
data = []
for sport in dataset['Sport']:
    ds = dataset[dataset['Sport']==sport]
    trace = go.Scatter(
        x = ds['Height'],
        y = ds['Weight'],
        name = sport,
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=10,
            size=ds['ID'],
            line=dict(
                width=2
            ),),
        text = ds['hover_text']
    )
    data.append(trace)

layout = go.Layout(
    title='Athlets height and weight mean - grouped by sport',
    xaxis=dict(
        title='Height [cm]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    yaxis=dict(
        title='Weight [kg]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(254, 254, 254)',
    showlegend=False,
)


fig = dict(data = data, layout = layout)

iplot(fig, filename='athlets_body_measures')

## SLIDER

In [148]:
tmp = olympics_df.groupby(['Sport', 'Year'])[HW].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sport', 'Year'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)
dataset.head(2)

,Sport,Year,Height,Weight,ID
0,Alpine Skiing,1936,169.250000,61.000000,103
1,Alpine Skiing,1948,170.116279,64.666667,360


In [154]:
df_olymp.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,271116.0,68248.954396,39022.286345,1.0,34643.0,68205.0,102097.25,135571.0
Age,261642.0,25.556898,6.393561,10.0,21.0,24.0,28.00,97.0
Height,210945.0,175.338970,10.518462,127.0,168.0,175.0,183.00,226.0
Weight,208241.0,70.702393,14.348020,25.0,60.0,70.0,79.00,214.0
Year,271116.0,1978.378480,29.877632,1896.0,1960.0,1988.0,2002.00,2016.0


In [149]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Year: {}<br>'+
                       'Sport: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Year'],
                                            row['Sport'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [161]:
years = (olympics_df.groupby(['Year'])['Year'].nunique()).index
sports = (olympics_df.groupby(['Sport'])['Sport'].nunique()).index
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [155, 195], 'title': 'Height'}
figure['layout']['yaxis'] = {'range': [25, 120],'title': 'Weight'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['showlegend'] = False
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1896',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
# make data
year = 1896
for sport in sports:
    dataset_by_year = dataset[dataset['Year'] == year]
    dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sport'] == sport]

    data_dict = {
        'x': list(dataset_by_year_and_season['Height']),
        'y': list(dataset_by_year_and_season['Weight']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_season['hover_text']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'size': list(dataset_by_year_and_season['ID'])
        },
        'name': sport
    }
    figure['data'].append(data_dict)
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for sport in sports:
        dataset_by_year = dataset[dataset['Year'] == int(year)]
        dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sport'] == sport]

        data_dict = {
            'x': list(dataset_by_year_and_season['Height']),
            'y': list(dataset_by_year_and_season['Weight']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_season['hover_text']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'size':  list(dataset_by_year_and_season['ID'])
            },
            'name': sport
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
figure['layout']['sliders'] = [sliders_dict]
iplot(figure)

In [163]:
# para haceer lo mismo pero agrupados por genero
tmp = olympics_df.groupby(['Sex'])[HW].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sex'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)

In [164]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sex: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Sex'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [165]:
data = []
for sex in dataset['Sex']:
    ds = dataset[dataset['Sex']==sex]
    trace = go.Scatter(
        x = ds['Height'],
        y = ds['Weight'],
        name = sex,
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=10,
            size=ds['ID'],
            line=dict(
                width=2
            ),),
        text = ds['hover_text']
    )
    data.append(trace)

layout = go.Layout(
    title='Athlets height and weight mean - grouped by Sex',
    xaxis=dict(
        title='Height [cm]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    yaxis=dict(
        title='Weight [kg]',
        gridcolor='rgb(128, 128, 128)',
        zerolinewidth=1,
        ticklen=1,
        gridwidth=0.5,
    ),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(254, 254, 254)',
    showlegend=False,
)


fig = dict(data = data, layout = layout)

iplot(fig, filename='athlets_body_measures2')

In [ ]:
# peso y altura de atletas a traves de los años

In [167]:
tmp = olympics_df.groupby(['Sex', 'Year'])[HW].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sex', 'Year'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)

In [168]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Year: {}<br>'+
                       'Sex: {}<br>'+
                      'Number of athlets: {}<br>'+
                      'Mean Height: {}<br>'+
                      'Mean Weight: {}<br>').format(row['Year'],
                                            row['Sex'],
                                            row['ID'],
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [173]:
years = (olympics_df.groupby(['Year'])['Year'].nunique()).index
sexes = (olympics_df.groupby(['Sex'])['Sex'].nunique()).index
# make figure
figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['xaxis'] = {'range': [160, 190], 'title': 'Height'}
figure['layout']['yaxis'] = {'range': [20, 150],'title': 'Weight'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['showlegend'] = False
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1896',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
# make data
year = 1896
for sex in sexes:
    dataset_by_year = dataset[dataset['Year'] == year]
    dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sex'] == sex]

    data_dict = {
        'x': list(dataset_by_year_and_season['Height']),
        'y': list(dataset_by_year_and_season['Weight']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_season['hover_text']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'size': list(dataset_by_year_and_season['ID'])
        },
        'name': sex
    }
    figure['data'].append(data_dict)
# make frames
for year in years:
    frame = {'data': [], 'name': str(year)}
    for sex in sexes:
        dataset_by_year = dataset[dataset['Year'] == int(year)]
        dataset_by_year_and_season = dataset_by_year[dataset_by_year['Sex'] == sex]

        data_dict = {
            'x': list(dataset_by_year_and_season['Height']),
            'y': list(dataset_by_year_and_season['Weight']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_season['hover_text']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'size':  list(dataset_by_year_and_season['ID'])
            },
            'name': sex
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
figure['layout']['sliders'] = [sliders_dict]
iplot(figure)

In [ ]:
#atheltes HW by sex and sport

In [175]:
HWA=['Height', 'Weight', 'Age']

In [178]:
tmp = olympics_df.groupby(['Sport', 'Sex'])[HWA].agg('mean').dropna()
df1 = pd.DataFrame(tmp).reset_index()
tmp2 = olympics_df.groupby(['Sport', 'Sex'])['ID'].count()
df2 = pd.DataFrame(tmp2).reset_index()
dataset = df1.merge(df2)
dataset.head(2)

,Sport,Sex,Height,Weight,Age,ID
0,Alpine Skiing,F,167.221001,62.640307,22.334609,3398
1,Alpine Skiing,M,177.891374,78.626035,23.758266,5431


In [179]:
hover_text = []
for index, row in dataset.iterrows():
    hover_text.append(('Sex: {}<br>'+
                       'Sport: {}<br>'
                       'Number of athlets: {}<br>'+
                       'Mean Age: {}<br>'
                       'Mean Height: {}<br>'+
                       'Mean Weight: {}<br>').format(row['Sex'],
                                            row['Sport'],
                                            row['ID'],
                                            round(row['Age'],2),
                                            round(row['Height'],2),
                                            round(row['Weight'],2)))
dataset['hover_text'] = hover_text

In [183]:
def plot_bubble_chart(dataset,title):
    data = []
    for sport in dataset['Sport']:
        ds = dataset[dataset['Sport']==sport]
        trace = go.Scatter(
            x = ds['Height'],
            y = ds['Weight'],
            name = sport,
            marker=dict(
                symbol='circle',
                sizemode='area',
                sizeref=50,
                size=np.power(ds['Age'],3),
                line=dict(
                    width=2
                ),),
            text = ds['hover_text']
        )
        data.append(trace)

    layout = go.Layout(
        title= title,
        xaxis=dict(
            title='Height [cm]',
            gridcolor='rgb(128, 128, 128)',
            zerolinewidth=1,
            ticklen=1,
            gridwidth=0.5,
            range=[150,200]
        ),
        yaxis=dict(
            title='Weight [kg]',
            gridcolor='rgb(128, 128, 128)',
            zerolinewidth=1,
            ticklen=1,
            gridwidth=0.5,
            range=[45,100]
        ),
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(254, 254, 254)',
        showlegend=False,
    )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='athlets_body_measures')

In [181]:
#female HWA by sport
dF = dataset[dataset['Sex']=='F']
plot_bubble_chart(dF,'Female athlets height and weight mean - grouped by sport')

In [184]:
dM = dataset[dataset['Sex']=='M']
plot_bubble_chart(dM,'Male athlets height and weight mean - grouped by sport')

## MEDALS

In [185]:
tmp = olympics_df.groupby(['Country', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()

In [186]:
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

def draw_map(dataset, title, colorscale):
    trace = go.Choropleth(
                locations = dataset['Country'],
                locationmode='country names',
                z = dataset['ID'],
                text = dataset['Country'],
                autocolorscale =False,
                reversescale = True,
                colorscale = colorscale,
                marker = dict(
                    line = dict(
                        color = 'rgb(0,0,0)',
                        width = 0.5)
                ),
                colorbar = dict(
                    title = 'Medals',
                    tickprefix = '')
            )
    data = [trace]
    layout = go.Layout(
        title = title,
        geo = dict(
            showframe = True,
            showlakes = False,
            showcoastlines = True,
            projection = dict(
                type = 'natural earth'
            )
        )
    )
    fig = dict( data=data, layout=layout )
    iplot(fig)

In [194]:
#gold medals
draw_map(dfG, "Countries with Gold Medals",'viridis')

In [188]:
#silver medals
draw_map(dfS, "Countries with Silver Medals",'Greys')

In [189]:
#Bronze medals
draw_map(dfB, 'Countries with Bronze Medals', 'Oranges')

In [195]:
#Medals by season
tmp = olympics_df.groupby(['Year', 'City','Season', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

In [196]:
traceG = go.Bar(
    x = dfG['Year'],y = dfG['ID'],
    name="Gold",
     marker=dict(
                color='gold',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text = dfG['City']+ " (" + dfG['Season'] + ")",
)
traceS = go.Bar(
    x = dfS['Year'],y = dfS['ID'],
    name="Silver",
    marker=dict(
                color='Grey',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['City']+ " (" + dfS['Season'] + ")",
)

traceB = go.Bar(
    x = dfB['Year'],y = dfB['ID'],
    name="Bronze",
    marker=dict(
                color='Brown',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfB['City']+ " (" + dfB['Season'] + ")",
)

data = [traceG, traceS, traceB]
layout = dict(title = 'Medals per Olympic edition',
          xaxis = dict(title = 'Year', showticklabels=True),
          yaxis = dict(title = 'Number of medals'),
          hovermode = 'closest',
          barmode='stack'
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')

In [197]:
# medals per sport
tmp = olympics_df.groupby(['Sport', 'Medal'])['ID'].agg('count').dropna()
df = pd.DataFrame(tmp).reset_index()
dfG = df[df['Medal']=='Gold']
dfS = df[df['Medal']=='Silver']
dfB = df[df['Medal']=='Bronze']

In [198]:
traceG = go.Bar(
    x = dfG['Sport'],y = dfG['ID'],
    name="Gold",
     marker=dict(
                color='gold',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text = dfG['Sport'],
    #orientation = 'h'
)
traceS = go.Bar(
    x = dfS['Sport'],y = dfS['ID'],
    name="Silver",
    marker=dict(
                color='Grey',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfS['Sport'],
    #orientation = 'h'
)

traceB = go.Bar(
    x = dfB['Sport'],y = dfB['ID'],
    name="Bronze",
    marker=dict(
                color='Brown',
                line=dict(
                    color='black',
                    width=1),
                opacity=0.5,
            ),
    text=dfB['Sport'],
   # orientation = 'h'
)

data = [traceG, traceS, traceB]
layout = dict(title = 'Medals per sport',
          xaxis = dict(title = 'Sport', showticklabels=True, tickangle=45,
            tickfont=dict(
                size=8,
                color='black'),),
          yaxis = dict(title = 'Number of medals'),
          hovermode = 'closest',
          barmode='stack',
          showlegend=False,
          width=900,
          height=600,
         )
fig = dict(data=data, layout=layout)
iplot(fig, filename='events-sports1')